<h1>Exploring AirBnb in Europe</h1>


<h3>Import Necessary Libraries and Modules</h3>

This cell imports essential libraries:

pandas:    For data manipulation and analysis.<br>
matplotlib.pyplot:     For plotting and visualization.<br>
datetime and timedelta:    To handle date and time calculations.<br>
os:            To interact with the operating system, allowing file path management <br>
ipywidgets: Provides tools to create interactive widgets in the Jupyter notebook. In this code, we use it to create a dropdown menu that lets users select a city, dynamically updating the visualizations based on the chosen city.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import os
import ipywidgets as widgets
from ipywidgets import interact

<h3>Define Cities and Setup Directory</h3>

current_directory stores the directory path to access files. <br>
cities defines a list of European cities to be analyzed. <br>
Each dictionary (listings, reviews) is used to store data for each city, helping us access and process data city by city.

In [ ]:
# Set the current working directory
current_directory = os.getcwd()

# List of cities (note that all city names here start with uppercase letters to match file names)
cities = ["Amsterdam", "Athens", "Barcelona", "Berlin", "Copenhagen", "Dublin", 
          "Lisbon", "London", "Madrid", "Paris", "Rome", "Venice", "Vienna"]

# Create dictionaries to store the dataframes for each city
listings = {}
reviews = {}

# Loop through each city and load the respective data files
for city in cities:
    listings_path = os.path.join(current_directory, f"{city}_listings.csv")
    reviews_path = os.path.join(current_directory, f"{city}_reviews.csv")
    
    listings[city] = pd.read_csv(listings_path)
    reviews[city] = pd.read_csv(reviews_path)

print(listings['Athens'].head())  # Prints the first few rows of Athens listings

<h3>Filter Data to the Last 12 Months</h3>

In [ ]:
cutoff_date = datetime.now() - timedelta(days=365)

def filter_last_12_months_inplace(df, date_column):
    df[date_column] = pd.to_datetime(df[date_column], errors='coerce')
    df.drop(df[df[date_column] < cutoff_date].index, inplace=True)

for city in cities:
    filter_last_12_months_inplace(reviews[city], 'date')


<h3>Determining population</h3>

In order to determine the population of these European cities I used the following data found on Statista https://www.statista.com/statistics/1101883/largest-european-cities/

Below we load and handle the data.

In [ ]:
population_path = os.path.join(current_directory, "largest-cities-in-europe-in-2023.xlsx")
population_df = pd.read_excel(population_path, sheet_name=1, skiprows=4)
population_df = population_df.drop(population_df.columns[0], axis=1)
population_df.columns = ['City', 'Population']
population_df['City'] = population_df['City'].apply(lambda x: x.split(" (")[0])
city_population = dict(zip(population_df['City'], population_df['Population']))


<h3> Calculate Listings Count and Density</h3>

In [ ]:
listings_count = {}
listings_density = {}

for city in cities:
    listings_count[city] = listings[city]['id'].nunique()
    population = city_population.get(city, 0)
    listings_density[city] = (listings_count[city] / population) * 1000 if population else None

listings_df = pd.DataFrame(listings_count.items(), columns=['City', 'Number of Listings'])
print(listings_df)

<h3>Visualize Listings per City</h3>

In [ ]:
plt.figure(figsize=(10, 6))
plt.bar(listings_df['City'], listings_df['Number of Listings'], color='skyblue')
plt.xlabel('City')
plt.ylabel('Number of Listings')
plt.title('Airbnb Listings per City')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


<h3>Visualize Listings Density</h3>

In [ ]:
density_df = pd.DataFrame({
    'City': cities,
    'Number of Listings': [listings_count[city] for city in cities],
    'Listings per 1,000 Inhabitants': [listings_density[city] for city in cities]
})

plt.figure(figsize=(10, 6))
plt.bar(density_df['City'], density_df['Listings per 1,000 Inhabitants'], color='skyblue')
plt.xlabel('City')
plt.ylabel('Listings per 1,000 Inhabitants')
plt.title('Airbnb Listings Density per City')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


<h3>Estimate Income and Activity Metrics</h3>

In [ ]:
count = {}
total_income = {}
for city in cities:
    total_reviews = listings[city]['number_of_reviews_ltm'].sum()
    bookings = total_reviews * 0.5
    nights_booked = bookings * 3
    mean_price = listings[city]['price'].mean()
    income_per_listing = mean_price * bookings
    total_income[city] = income_per_listing
    count[city] = listings[city].shape[0]

average_income_per_listing = {city: total_income[city] / count[city] for city in cities}
income_df = pd.DataFrame({
    'City': cities,
    'Average Income per Listing (USD)': [average_income_per_listing[city] for city in cities]
})
print(income_df)


<h3>Visualisation</h3>

An attempt to replicate the visualisaton found in Inside Airbnb, using the drop-down menu you can select the city to view its respective statistics

In [ ]:
# Define the dropdown widget for selecting a city
city_dropdown = widgets.Dropdown(
    options=cities,
    value=cities[0],
    description='Select City:',
)

# Define the function to update the plots and tables based on the selected city
def update_plots_and_table(city):
    city_listings = listings[city]

    # --- Room Type Distribution ---
    room_type_counts = city_listings['room_type'].value_counts()
    room_type_labels = room_type_counts.index
    room_type_values = room_type_counts.values

    # Plot Room Type Distribution
    plt.figure(figsize=(8, 5))
    plt.bar(room_type_labels, room_type_values, color="skyblue")
    plt.xlabel("Room Type")
    plt.ylabel("Number of Listings")
    plt.title(f"Room Type Distribution for {city}")
    plt.tight_layout()
    plt.show()

    # --- Listings per Host ---
    # Calculate the number of listings per host
    listings_per_host = city_listings['host_id'].value_counts()

    # Categorize listings per host into 1, 2, ..., 9, and 10+
    listings_per_host = listings_per_host.apply(lambda x: x if x < 10 else 10)

    # Count the occurrences of each category
    listings_distribution = listings_per_host.value_counts().sort_index()

    # Define the custom x-axis categories as strings
    categories = [str(i) for i in range(1, 10)] + ["10+"]
    distribution_values = [listings_distribution.get(i, 0) for i in range(1, 10)] + [listings_distribution.get(10, 0)]

    # Plot Listings per Host
    plt.figure(figsize=(10, 5))
    plt.bar(categories, distribution_values, color="skyblue")
    plt.xlabel("Listings per Host")
    plt.ylabel("Number of Hosts")
    plt.title(f"Listings per Host Distribution for {city}")
    plt.tight_layout()
    plt.show()

    # --- Top 50 Hosts Table ---
    # Calculate the number of listings per host for the current city
    host_listings_count = city_listings['host_name'].value_counts()

    # Get the top 50 hosts with the most listings for the current city
    top_50_hosts = host_listings_count.head(50).index

    # Prepare a list to hold the data for each of the top 50 hosts
    host_data = []

    for host in top_50_hosts:
        host_listings = city_listings[city_listings['host_name'] == host]

        # Count the number of each room type for the host
        room_type_counts = host_listings['room_type'].value_counts()

        # Reindex the room type counts to ensure all room types are present
        room_type_counts = room_type_counts.reindex(['Entire home/apt', 'Private room', 'Shared room', 'Hotel room'], fill_value=0)

        # Get the total number of listings for the host
        total_listings = host_listings.shape[0]

        # Append the data to the list
        host_data.append([host] + room_type_counts.tolist() + [total_listings])

    # Create the DataFrame for the current city with the desired structure
    columns = ['Host Name', 'Entire home/apt', 'Private room', 'Shared room', 'Hotel room', 'Total Listings']
    host_df = pd.DataFrame(host_data, columns=columns)

    # Display the table for the current city
    display(host_df.style.set_table_styles([
        {'selector': 'thead th', 'props': [('background-color', 'lightblue'), ('font-weight', 'bold')]},
        {'selector': 'tbody td', 'props': [('text-align', 'center')]},
    ]))
    print("\n" + "="*50)  # Separator between city tables

    # --- Activity Graph ---
    # Define occupancy bins
    bins = [0, 1, 30, 60, 90, 100]
    labels = ["0%", "1-30%", "31-60%", "61-90%", "91-100%"]

    # Calculate bookings, nights booked, and occupancy for each listing
    city_listings['bookings'] = city_listings['number_of_reviews_ltm'] * 0.5
    city_listings['nights_booked'] = city_listings['bookings'] * 3

    # Occupancy as percentage of total nights in a year
    city_listings['occupancy_percentage'] = (city_listings['nights_booked'] / 365) * 100

    # Bin the occupancy percentages into specified ranges
    city_listings['occupancy_range'] = pd.cut(city_listings['occupancy_percentage'], bins=bins, labels=labels, right=False)
    occupancy_distribution = city_listings['occupancy_range'].value_counts().reindex(labels).fillna(0)

    # Plot Activity Graph
    plt.figure(figsize=(8, 5))
    plt.bar(occupancy_distribution.index, occupancy_distribution.values, color="skyblue")
    plt.xlabel("Occupancy Range (%)")
    plt.ylabel("Number of Listings")
    plt.title(f"Activity Graph for {city}")
    plt.tight_layout()
    plt.show()

# Link the dropdown menu to the update function
interact(update_plots_and_table, city=city_dropdown)
